<a href="https://colab.research.google.com/github/BergerPerkins/Vehicle-Counting/blob/main/Tracking_and_Counting_vehicles_using_YOLOv8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Using Ultralytics YOLOv8, ByteTrack and Supervision for tracking and counting**
Detection with yolov8

Tracking with BoT-SORT

Counting using supervision

In [1]:
#!pip install ultralytics

import ultralytics
ultralytics.__version__

'8.0.218'

In [2]:
#!pip install supervision==0.14.0

import supervision
print("supervision.__version__:", supervision.__version__)

supervision.__version__: 0.14.0


In [3]:
import torch
torch.__version__

'2.1.0+cu118'

# **Detection**

In [5]:
from ultralytics import YOLO

# Load a pretrained YOLOv8n model
model = YOLO('yolov8n.pt')

# Run inference on 'bus.jpg' with arguments
model.predict(source="d.mp4", save=True, imgsz=320, conf=0.5)

100%|██████████| 6.23M/6.23M [00:00<00:00, 297MB/s]




WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (1/687) /content/d.mp4: 192x320 3 cars, 216.6ms
video 1/1 (2/687) /content/d.mp4: 192x320 3 cars, 8.7ms
video 1/1 (3/687) /content/d.mp4: 192x320 4 cars, 11.9ms
video 1/1 (4/687) /content/d.mp4: 192x320 4 cars, 12.7ms
video 1/1 (5/687) /content/d.mp4: 192x320 4 cars, 9.0ms
video 1/1 (6/687) /content/d.mp4: 192x320 3 cars, 7.8ms
video 1/1 (7/687) /content/d.mp4: 192x320 4 cars, 7.9ms
video 1/1 (8/687) /content/d.mp4: 192x320 5 cars, 8.0ms
vide

[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted p

# **Tracking**

In [7]:
from ultralytics import YOLO

# Configure the tracking parameters and run the tracker
model = YOLO('yolov8n.pt')

results = model.track(source="d.mp4",conf=0.3, iou=0.5, save=True, tracker="bytetrack.yaml")



WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (1/687) /content/d.mp4: 384x640 7 cars, 245.6ms
video 1/1 (2/687) /content/d.mp4: 384x640 7 cars, 16.5ms
video 1/1 (3/687) /content/d.mp4: 384x640 7 cars, 10.1ms
video 1/1 (4/687) /content/d.mp4: 384x640 7 cars, 29.1ms
video 1/1 (5/687) /content/d.mp4: 384x640 7 cars, 12.1ms
video 1/1 (6/687) /content/d.mp4: 384x640 7 cars, 24.0ms
video 1/1 (7/687) /content/d.mp4: 384x640 7 cars, 12.2ms
video 1/1 (8/687) /content/d.mp4: 384x640 7 cars, 15.2ms

# **Tracking and counting - 1 line**

In [4]:
import cv2
from collections import defaultdict
import supervision as sv
from ultralytics import YOLO

# Load the YOLOv8 model
model = YOLO('yolov8n.pt')

# Set up video capture
cap = cv2.VideoCapture("d.mp4")

# Define the line coordinates
START = sv.Point(182, 254)
END = sv.Point(462, 254)


# Store the track history
track_history = defaultdict(lambda: [])

# Create a dictionary to keep track of objects that have crossed the line
crossed_objects = {}

# Open a video sink for the output video
video_info = sv.VideoInfo.from_video_path("d.mp4")
with sv.VideoSink("output_single_line.mp4", video_info) as sink:

    while cap.isOpened():
        success, frame = cap.read()

        if success:
            # Run YOLOv8 tracking on the frame, persisting tracks between frames
            results = model.track(frame, classes=[2, 3, 5, 7], persist=True, save=True, tracker="bytetrack.yaml")

            # Get the boxes and track IDs
            boxes = results[0].boxes.xywh.cpu()
            track_ids = results[0].boxes.id.int().cpu().tolist()

            # Visualize the results on the frame
            annotated_frame = results[0].plot()
            detections = sv.Detections.from_yolov8(results[0])

            # Plot the tracks and count objects crossing the line
            for box, track_id in zip(boxes, track_ids):
                x, y, w, h = box
                track = track_history[track_id]
                track.append((float(x), float(y)))  # x, y center point
                if len(track) > 30:  # retain 30 tracks for 30 frames
                    track.pop(0)

                # Check if the object crosses the line
                if START.x < x < END.x and abs(y - START.y) < 5:  # Assuming objects cross horizontally
                    if track_id not in crossed_objects:
                        crossed_objects[track_id] = True

                    # Annotate the object as it crosses the line
                    cv2.rectangle(annotated_frame, (int(x - w / 2), int(y - h / 2)), (int(x + w / 2), int(y + h / 2)), (0, 255, 0), 2)

            # Draw the line on the frame
            cv2.line(annotated_frame, (START.x, START.y), (END.x, END.y), (0, 255, 0), 2)

            # Write the count of objects on each frame
            count_text = f"Objects crossed: {len(crossed_objects)}"
            cv2.putText(annotated_frame, count_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

            # Write the frame with annotations to the output video
            sink.write_frame(annotated_frame)
        else:
            break

# Release the video capture
cap.release()


0: 384x640 7 cars, 114.1ms
Speed: 3.1ms preprocess, 114.1ms inference, 3.1ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/detect/predict8

0: 384x640 7 cars, 6.5ms
Speed: 2.9ms preprocess, 6.5ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/detect/predict8

0: 384x640 7 cars, 7.5ms
Speed: 2.1ms preprocess, 7.5ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/detect/predict8

0: 384x640 7 cars, 6.9ms
Speed: 1.9ms preprocess, 6.9ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/detect/predict8

0: 384x640 7 cars, 8.4ms
Speed: 1.5ms preprocess, 8.4ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/detect/predict8

0: 384x640 7 cars, 6.8ms
Speed: 1.9ms preprocess, 6.8ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/detect/predict8



<ipython-input-4-779eb8431fd1>:40: DeprecationWarning: from_yolov8 is deprecated: 
        This method is deprecated and removed in 0.15.0 release.
        Use sv.Detections.from_ultralytics() instead as it is more generic and
        can be used for detections from any ultralytics.engine.results.Results Object
        
  detections = sv.Detections.from_yolov8(results[0])


0: 384x640 7 cars, 10.7ms
Speed: 1.4ms preprocess, 10.7ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/detect/predict8

0: 384x640 7 cars, 7.8ms
Speed: 1.7ms preprocess, 7.8ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/detect/predict8

0: 384x640 7 cars, 7.4ms
Speed: 1.3ms preprocess, 7.4ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/detect/predict8

0: 384x640 7 cars, 9.5ms
Speed: 1.1ms preprocess, 9.5ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/detect/predict8

0: 384x640 7 cars, 6.7ms
Speed: 1.7ms preprocess, 6.7ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/detect/predict8

0: 384x640 7 cars, 9.4ms
Speed: 1.1ms preprocess, 9.4ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/detect/predict8

0: 384x640 7 cars, 6.9ms
Speed: 1.3ms 

# **Tracking and counting - 2 line**

In [13]:
import cv2
from collections import defaultdict
import supervision as sv
from ultralytics import YOLO

# Load the YOLOv8 model
model = YOLO('yolov8n.pt')

# Set up video capture
cap = cv2.VideoCapture("d.mp4")

# Define the line coordinates
START = sv.Point(182, 254)
END = sv.Point(462, 254)

# Define the side line coordinates
START_sideline = sv.Point(198, 250)
END_sideline = sv.Point(238, 202)

# Store the track history
track_history = defaultdict(lambda: [])

# Create a dictionary to keep track of objects that have crossed the line
crossed_objects_main = {}
# Create a dictionary to keep track of objects that have crossed the sideline
crossed_objects_sideline = {}

# Open a video sink for the output video
video_info = sv.VideoInfo.from_video_path("d.mp4")
with sv.VideoSink("output_2_line.mp4", video_info) as sink:

    while cap.isOpened():
        success, frame = cap.read()

        if success:
            # Run YOLOv8 tracking on the frame, persisting tracks between frames
            results = model.track(frame, classes=[2, 3, 5, 7], persist=True, save=True, tracker="bytetrack.yaml")

            # Get the boxes and track IDs
            boxes = results[0].boxes.xywh.cpu()
            track_ids = results[0].boxes.id.int().cpu().tolist()

            # Visualize the results on the frame
            annotated_frame = results[0].plot()
            detections = sv.Detections.from_yolov8(results[0])

            # Plot the tracks and count objects crossing the line
            for box, track_id in zip(boxes, track_ids):
                x, y, w, h = box
                track = track_history[track_id]
                track.append((float(x), float(y)))  # x, y center point
                if len(track) > 30:  # retain 30 tracks for 30 frames
                    track.pop(0)

                # Check if the object crosses the line
                if START.x < x < END.x and abs(y - START.y) < 5:  # Assuming objects cross horizontally
                    if track_id not in crossed_objects_main:
                        crossed_objects_main[track_id] = True

                # Check if the object crosses the sideline
                if START_sideline.x < x < END_sideline.x and min(START_sideline.y, END_sideline.y) < y < max(START_sideline.y, END_sideline.y):
                    if track_id not in crossed_objects_sideline:
                        crossed_objects_sideline[track_id] = True

                    # Annotate the object as it crosses the line
                    cv2.rectangle(annotated_frame, (int(x - w / 2), int(y - h / 2)), (int(x + w / 2), int(y + h / 2)), (0, 255, 0), 2)

            # Draw the line on the frame
            cv2.line(annotated_frame, (START.x, START.y), (END.x, END.y), (0, 255, 0), 2)
            cv2.line(annotated_frame, (START_sideline.x, START_sideline.y), (END_sideline.x, END_sideline.y), (255, 0, 0), 2)

            # Write the count of objects on each frame
            count_main_text = f"Objects crossed mainline: {len(crossed_objects_main)}"
            count_sideline_text=f"objects crossed sideline: {len(crossed_objects_sideline)}"

            cv2.putText(annotated_frame, count_main_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            cv2.putText(annotated_frame, count_sideline_text, (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)


            # Write the frame with annotations to the output video
            sink.write_frame(annotated_frame)
        else:
            break

# Release the video capture
cap.release()


0: 384x640 7 cars, 10.2ms
Speed: 1.7ms preprocess, 10.2ms inference, 2.2ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/detect/predict11

0: 384x640 7 cars, 10.1ms
Speed: 1.3ms preprocess, 10.1ms inference, 2.1ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/detect/predict11

0: 384x640 7 cars, 10.1ms
Speed: 1.3ms preprocess, 10.1ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/detect/predict11

0: 384x640 7 cars, 8.6ms
Speed: 1.5ms preprocess, 8.6ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/detect/predict11

0: 384x640 7 cars, 8.7ms
Speed: 1.3ms preprocess, 8.7ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/detect/predict11



<ipython-input-13-438ff3c43d31>:45: DeprecationWarning: from_yolov8 is deprecated: 
        This method is deprecated and removed in 0.15.0 release.
        Use sv.Detections.from_ultralytics() instead as it is more generic and
        can be used for detections from any ultralytics.engine.results.Results Object
        
  detections = sv.Detections.from_yolov8(results[0])


0: 384x640 7 cars, 10.9ms
Speed: 1.4ms preprocess, 10.9ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/detect/predict11

0: 384x640 7 cars, 7.0ms
Speed: 2.1ms preprocess, 7.0ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/detect/predict11

0: 384x640 7 cars, 7.8ms
Speed: 1.4ms preprocess, 7.8ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/detect/predict11

0: 384x640 7 cars, 7.4ms
Speed: 1.4ms preprocess, 7.4ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/detect/predict11

0: 384x640 7 cars, 14.5ms
Speed: 1.2ms preprocess, 14.5ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/detect/predict11

0: 384x640 7 cars, 6.7ms
Speed: 1.7ms preprocess, 6.7ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/detect/predict11

0: 384x640 7 cars, 7.9ms
Speed